In [5]:
import numpy as np
import pandas as pd
import sys
from binance_futures_client import BinanceFuturesClient
import utils


In [6]:
client = BinanceFuturesClient(None, None, testing=True)

In [8]:
utils.ms_to_datetime(client.get_server_time())

datetime.datetime(2024, 4, 30, 16, 27, 41, 982000)

In [15]:
candles = client.klines('BTCUSDT', '4h', limit=300)

In [16]:
df = pd.DataFrame(candles)

In [18]:
df.describe()

,open_time,open,high,low,close,volume,close_time
count,3.000000e+02,300.000000,300.000000,300.000000,300.000000,300.000000,3.000000e+02
mean,1.712311e+12,67980.112667,69022.657000,67015.343000,67942.058000,6893.148013,1.712326e+12
std,1.249153e+09,3226.145183,3793.744426,3262.564385,3266.836523,7184.626305,1.249153e+09
min,1.710158e+12,61440.500000,61800.000000,59299.000000,61430.700000,398.197000,1.710173e+12
25%,1.711235e+12,65207.650000,66187.500000,64370.150000,65339.225000,1528.889000,1.711249e+12
50%,1.712311e+12,67773.400000,68952.900000,66673.550000,67685.700000,3198.430500,1.712326e+12
75%,1.713388e+12,70709.150000,71499.925000,70006.200000,70738.375000,12254.685250,1.713402e+12
max,1.714464e+12,75378.800000,100130.100000,74290.100000,75330.000000,33435.313000,1.714478e+12


In [20]:
df.head(5)

,open_time,open,high,low,close,volume,close_time
0,1710158400000,71300.2,72339.5,70801.1,72232.9,19032.636,1710172799999
1,1710172800000,72129.0,72799.0,71803.3,72000.0,20309.213,1710187199999
2,1710187200000,72000.0,72499.8,71950.1,72002.2,16368.742,1710201599999
3,1710201600000,72153.8,72397.2,71692.0,72232.9,18253.165,1710215999999
4,1710216000000,71992.0,72489.9,71509.8,72232.9,17106.055,1710230399999


In [21]:
utils.ms_to_datetime(df[:, 0])

InvalidIndexError: (slice(None, None, None), 0)

In [22]:
df[:, 0]

InvalidIndexError: (slice(None, None, None), 0)

In [25]:
# get the open_time column
open_time = df.iloc[:, 0]
open_time.head(5)

0    1710158400000
1    1710172800000
2    1710187200000
3    1710201600000
4    1710216000000
Name: open_time, dtype: int64

In [30]:
# add a column with the open_time converted to datetime
df['open_dt'] = pd.to_datetime(df.iloc[:, 0], unit='ms')
df.head(5)

,open_time,open,high,low,close,volume,close_time,open_dt
0,1710158400000,71300.2,72339.5,70801.1,72232.9,19032.636,1710172799999,2024-03-11 12:00:00
1,1710172800000,72129.0,72799.0,71803.3,72000.0,20309.213,1710187199999,2024-03-11 16:00:00
2,1710187200000,72000.0,72499.8,71950.1,72002.2,16368.742,1710201599999,2024-03-11 20:00:00
3,1710201600000,72153.8,72397.2,71692.0,72232.9,18253.165,1710215999999,2024-03-12 00:00:00
4,1710216000000,71992.0,72489.9,71509.8,72232.9,17106.055,1710230399999,2024-03-12 04:00:00


In [37]:
# add a column as moving average of the close price
slow_ma = df.iloc[:, 4].rolling(window=10).mean()
fast_ma = df.iloc[:, 4].rolling(window=5).mean()


In [43]:
def get_long_position(symbol):
    positions = client.get_open_positions()
    long_positions = [pos for pos in positions if (pos['positionSide'] == 'LONG' and pos['symbol'] == symbol)]
    return None if len(long_positions) == 0 else long_positions[0]

pos = get_long_position('BTCUSDT')
print(pos)

{'symbol': 'BTCUSDT', 'initialMargin': '37.01538000', 'maintMargin': '2.96123040', 'unrealizedProfit': '-0.98760000', 'positionInitialMargin': '37.01538000', 'openOrderInitialMargin': '0', 'leverage': '20', 'isolated': False, 'entryPrice': '61774.6', 'breakEvenPrice': '61799.30984', 'maxNotional': '20000000', 'positionSide': 'LONG', 'positionAmt': '0.012', 'notional': '740.30760000', 'isolatedWallet': '0', 'updateTime': 1714474669538, 'bidNotional': '0', 'askNotional': '0'}


In [49]:
#initialize a new dataframe for the moving averages
df_ma = pd.DataFrame()
df_ma['slow_ma'] = df['close'].rolling(window=21).mean()
# display df_ma from index 19 to 25, only 'slow_ma' column
df_ma.loc[19:25, 'slow_ma']



19             NaN
20    72439.338095
21    72415.642857
22    72382.600000
23    72290.257143
24    72216.657143
25    72143.085714
Name: slow_ma, dtype: float64

In [52]:
ma_frame = pd.DataFrame()
ma_frame['ma'] = df['close'].rolling(window=21).mean()
ma_frame.loc[19:25]


,ma
19,NaN
20,72439.338095
21,72415.642857
22,72382.600000
23,72290.257143
24,72216.657143
25,72143.085714
